In [4]:
import os
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_validate

import kagglehub

from constants import dataset_name, opciones, CV, scoring
from utils import cargar_imagenes, evaluar_rendimiento, save_excel_cv

from modelos.RegresionLogistica import regresion_logistica
from modelos.CNN import cnn1, cnn2
from modelos.KNN import knn, knn_with_gridsearch
from modelos.ArbolDeDecision import arbol_decision, arbol_decision_vgg16, arbol_vgg16_pca
from modelos.MobileNetV2 import train_mobilenetv2_model
from modelos.RandomForest import random_forest, rforest_vgg16_pca, rforest_vgg16_pca_hog

from tensorflow.keras.utils import to_categorical
from modelos.MobileNetV2 import train_mobilenetv2_model

import matplotlib.pyplot as plt

# Descargar dataset
path = kagglehub.dataset_download(dataset_name)

# cargar train y test
X_train_rgb, y_train = cargar_imagenes(os.path.join(path, 'flowers/flowers/flower_photos/train'), channel_mode="rgb")
X_test_rgb, y_test = cargar_imagenes(os.path.join(path, 'flowers/flowers/flower_photos/test'), channel_mode="rgb")
X_train_gray, _ = cargar_imagenes(os.path.join(path, 'flowers/flowers/flower_photos/train'), channel_mode="grayscale")
X_test_gray, _ = cargar_imagenes(os.path.join(path, 'flowers/flowers/flower_photos/test'), channel_mode="grayscale")
X_train_rgb_64, _ = cargar_imagenes(os.path.join(path, 'flowers/flowers/flower_photos/train'), target_size=(64, 64))
X_test_rgb_64, _ = cargar_imagenes(os.path.join(path, 'flowers/flowers/flower_photos/test'), target_size=(64, 64))

# Codificar etiquetas
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

epochs = 100
batch_size = 100
history = []


KeyboardInterrupt: 

In [2]:
## si es regresión logística rgb ##
modelLR_rgb, scoresLR_rgb = regresion_logistica(X_train_rgb, y_train_encoded)
save_excel_cv(scoresLR_rgb, "Regresión Logística RGB")

In [13]:
print(scoresLR_rgb)

{'fit_time': array([243.75289154, 238.94802904, 223.00812745, 309.87033486,
       195.86208987]), 'score_time': array([0.65084386, 0.56014276, 0.59805107, 0.5166657 , 0.53463554]), 'test_accuracy': array([0.37429379, 0.34887006, 0.32627119, 0.37429379, 0.37711864]), 'test_precision': array([0.360911  , 0.34869617, 0.32552323, 0.36698258, 0.37940236]), 'test_recall': array([0.3612659 , 0.34811868, 0.3235971 , 0.35968507, 0.36655325]), 'test_f1': array([0.35879968, 0.34641024, 0.32419283, 0.35815078, 0.36552693]), 'test_roc_auc': array([0.66949605, 0.66420851, 0.64494095, 0.68413902, 0.68129405])}


In [5]:
## si es regresión logística gray ##
modelLR_gray, scoresLR_gray = regresion_logistica(X_train_gray, y_train_encoded)
save_excel_cv(scoresLR_gray, "Regresión Logística gray")


In [6]:
## si es CNN1 ##
model1 = cnn1()
model1.summary()

history.append(model1.fit(X_train_rgb_64, y_train_encoded,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(X_test_rgb_64, y_test_encoded)))

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 64)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,877 (93.27 KB)

 Trainable params: 23,877 (93.27 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 115ms/step - accuracy: 0.2500 - loss: 1.5903 - val_accuracy: 0.3800 - val_loss: 1.5105
Epoch 2/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 110ms/step - accuracy: 0.3677 - loss: 1.4486 - val_accuracy: 0.3400 - val_loss: 1.3872
Epoch 3/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 109ms/step - accuracy: 0.3971 - loss: 1.3413 - val_accuracy: 0.3600 - val_loss: 1.3305
Epoch 4/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 110ms/step - accuracy: 0.4175 - loss: 1.2966 - val_accuracy: 0.4800 - val_loss: 1.2612
Epoch 5/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 108ms/step - accuracy: 0.4527 - loss: 1.2624 - val_accuracy: 0.4600 - val_loss: 1.2378
Epoch 6/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 114ms/step - accuracy: 0.4683 - loss: 1.2587 - val_accuracy: 0.5000 - val_loss: 1.1928
Epoch 7/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 110ms/step - accuracy: 0.4943 - loss: 1.2164 - val_accuracy: 0.5000 - val_loss: 1.1920
Epoch 8/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 109ms/step - accuracy: 0.4974 - loss: 1.2165 - val_accu

In [ ]:
## si es CNN2 ##    
model2 = cnn2()
model2.summary()

history.append(model2.fit(X_train_rgb_64, y_train_encoded,
                epochs=epochs,
                batch_size=batch_size,
                validation_data=(X_test_rgb_64, y_test_encoded)))

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 64, 64, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64, 64, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 32, 32, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 64)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 70,437 (275.14 KB)

 Trainable params: 70,245 (274.39 KB)

 Non-trainable params: 192 (768.00 B)

Epoch 1/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 23s 545ms/step - accuracy: 0.4161 - loss: 1.4247 - val_accuracy: 0.3000 - val_loss: 1.5844
Epoch 2/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 20s 553ms/step - accuracy: 0.5689 - loss: 1.0896 - val_accuracy: 0.4200 - val_loss: 1.5080
Epoch 3/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 20s 554ms/step - accuracy: 0.6341 - loss: 0.9761 - val_accuracy: 0.4600 - val_loss: 1.4856
Epoch 4/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 20s 551ms/step - accuracy: 0.6528 - loss: 0.9130 - val_accuracy: 0.2200 - val_loss: 1.5359
Epoch 5/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 20s 556ms/step - accuracy: 0.6766 - loss: 0.8722 - val_accuracy: 0.3000 - val_loss: 1.5017
Epoch 6/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 20s 557ms/step - accuracy: 0.6867 - loss: 0.8269 - val_accuracy: 0.3000 - val_loss: 1.4924
Epoch 7/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 20s 560ms/step - accuracy: 0.7261 - loss: 0.7307 - val_accuracy: 0.3000 - val_loss: 1.5482
Epoch 8/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 20s 560ms/step - accuracy: 0.7003 - loss: 0.7604 - 

In [8]:
## KNN ## 
_, _, scoresKNN = knn(X_train_rgb, y_train_encoded, X_test_rgb, y_test_encoded)

save_excel_cv(scoresKNN, "KNN")

TypeError: knn() takes 3 positional arguments but 4 were given

In [2]:
modelKNN, pcaKNN, scoresKNN = knn_with_gridsearch(X_train_rgb_64, y_train_encoded, X_test_rgb_64, y_test_encoded)

Mejores parámetros: {'knn__n_neighbors': 7, 'pca__n_components': 30}
   param_pca__n_components  param_knn__n_neighbors  mean_test_accuracy  \
2                       30                       7            0.435311   
3                       50                       7            0.430226   
0                       30                       3            0.413842   
1                       50                       3            0.412429   

   mean_test_precision  mean_test_recall  mean_test_f1  mean_test_roc_auc  
2             0.436485          0.421142      0.416222           0.728680  
3             0.440036          0.412501      0.406571           0.725369  
0             0.425091          0.403926      0.402425           0.687863  
1             0.424728          0.400201      0.397598           0.686073  


In [3]:
model_tree = arbol_decision(X_train_rgb_64, y_train_encoded)
evaluar_rendimiento(model_tree, X_test_rgb_64, y_test, "Árbol de Decisión")

KeyboardInterrupt: 

In [ ]:
model_tree = arbol_decision_vgg16(X_train_rgb_64, y_train_encoded)
evaluar_rendimiento(model_tree, X_test_rgb_64, y_test, "Árbol de Decisión VGG16")


In [ ]:
model_tree = arbol_vgg16_pca(X_train_rgb_64, y_train_encoded)
# Evaluar el modelo con las funciones definidas previamente
evaluar_rendimiento(model_tree, X_test_rgb_64, y_test, "Árbol de Decisión VGG16 PCA")

In [9]:
num_classes = len(np.unique(y_train_encoded))  # Asegurar que tenemos el número correcto de clases
y_train_onehot = to_categorical(y_train_encoded, num_classes)
y_test_onehot = to_categorical(y_test_encoded, num_classes)

# Llamada a la función de entrenamiento
model_1 = train_mobilenetv2_model(X_train_rgb, y_train_onehot, X_test_rgb, y_test_onehot, epochs=10, batch_size=32, dense_units=1024, dropout_rate=0.5, learning_rate=0.001)
model_2 = train_mobilenetv2_model(X_train_rgb_64, y_train_onehot, X_test_rgb_64, y_test_onehot, input_shape=(64, 64, 3), epochs=10, batch_size=32, dense_units=1024, dropout_rate=0.5, learning_rate=0.001)
model_3 = train_mobilenetv2_model(X_train_rgb_64, y_train_onehot, X_test_rgb_64, y_test_onehot, input_shape=(64, 64, 3),epochs=10, batch_size=32, dense_units=1024, dropout_rate=0.5, learning_rate=0.0001)
model_4 = train_mobilenetv2_model(X_train_rgb_64, y_train_onehot, X_test_rgb_64, y_test_onehot, input_shape=(64, 64, 3),epochs=10, batch_size=32, dense_units=1024, dropout_rate=0.3, learning_rate=0.001)

d:\Documentos\Master Informatica Industrial y Robotica\VA I\Trabajo_git\clasificacion-supervisada\modelos\MobileNetV2.py:13: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)


Epoch 1/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 95s 771ms/step - accuracy: 0.7031 - loss: 0.8184 - val_accuracy: 0.8000 - val_loss: 0.7273
Epoch 2/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 83s 747ms/step - accuracy: 0.9016 - loss: 0.2687 - val_accuracy: 0.7600 - val_loss: 0.7518
Epoch 3/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 79s 712ms/step - accuracy: 0.9156 - loss: 0.2395 - val_accuracy: 0.8200 - val_loss: 0.7202
Epoch 4/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 77s 696ms/step - accuracy: 0.9380 - loss: 0.1680 - val_accuracy: 0.7800 - val_loss: 0.8725
Epoch 5/10
 79/111 ━━━━━━━━━━━━━━━━━━━━ 22s 714ms/step - accuracy: 0.9434 - loss: 0.1593

KeyboardInterrupt: 

In [11]:
modelRF, scoresRF = random_forest(X_train_rgb_64, y_train, X_test_rgb_64)
save_excel_cv(scoresRF, "RANDOM FOREST")

In [12]:
rf_model, scoresRFVGGPCA = rforest_vgg16_pca(X_train_rgb, y_train, X_test_rgb)        
save_excel_cv(scoresRF, "RANDOM FOREST VGG16 PCA")

111/111 ━━━━━━━━━━━━━━━━━━━━ 687s 6s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 3s/step


In [ ]:
rf_model, scoresRFHOG = rforest_vgg16_pca_hog(X_train_rgb_64, y_train, X_test_rgb_64)
save_excel_cv(scoresRFHOG, "RANDOM FOREST VGG16 PCA")

In [ ]:
## diagrama de cajas validación cru
data = [scoresLR_rgb['test_accuracy'], scoresLR_gray['test_accuracy'], scoresKNN['test_accuracy']]
_, ax = plt.subplots()
ax.set_title('Modelos')
ax.boxplot(data,labels=['LR_RGB','LR_GRAY','KNN'])